![](https://europe-west1-atp-views-tracker.cloudfunctions.net/working-analytics?notebook=tutorials--agent-with-tavily-web-access--search-extract-crawl)

# 1. ウェブの検索、抽出、クロール 🌐

ようこそ！このチュートリアルでは、Tavily APIのコア機能（セマンティックな理解を伴うウェブ検索、ライブウェブページからのコンテンツ抽出、ウェブサイト全体のクロール）を実際に体験します。

これらのスキルは、インターネットから最新の関連情報を必要とするAIエージェントやアプリケーションを構築する人にとって不可欠です。リアルタイムのウェブデータにプログラムでアクセスして処理する方法を学ぶことで、静的な言語モデルとそれらが動作する動的な世界との間のギャップを埋めることができ、エージェントをよりスマートで、より正確で、コンテキストを理解できるものにすることができます。

以下をカバーします：
- ウェブ検索を実行し、最も関連性の高い結果を取得する方法
- 任意のURLからクリーンで使用可能なコンテンツを抽出する方法
- ウェブサイトをクロールして包括的な情報を収集する方法
- 高度なパラメータでクエリを微調整する方法
 

---

## はじめに

以下の手順でセットアップします：

1. [app.tavily.com](https://app.tavily.com/home/?utm_source=github&utm_medium=referral&utm_campaign=nir_diamant)でTavilyに**サインアップ**してAPIキーを取得します。

   *ステップバイステップのガイダンスについては、以下にリンクされているスクリーンショットを参照してください：*

   - ![スクリーンショット：サインアップページ](assets/sign-up.png)
   - ![スクリーンショット：Tavily APIキーダッシュボード](assets/api-key.png)


2. Tavilyアカウントのダッシュボードから**APIキーをコピー**します。

3. 以下のセルに**APIキーを貼り付けて**、セルを実行します。

In [ ]:
# APIキーを.envファイルにエクスポートするには、以下のセルを実行します（実際のキーに置き換えてください）：
!echo "TAVILY_API_KEY=<your-tavily-api-key>" >> .env

以下のセルで依存関係をインストールします。

In [ ]:
%pip install --upgrade tavily-python --quiet

### Tavily APIクライアントのセットアップ

以下のコードは、APIキーを使用してTavilyクライアントをインスタンス化します。

In [ ]:
import os
import getpass
from dotenv import load_dotenv
from tavily import TavilyClient

# .envファイルから環境変数を読み込む
load_dotenv()

# 環境にまだ設定されていない場合は、ユーザーにAPIキーの入力を安全に促す
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

# 読み込まれたまたは提供されたAPIキーを使用してTavily APIクライアントを初期化
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

## 検索 🔍 

NYCに関する最新情報を取得するための基本的なウェブ検索クエリを実行してみましょう。

In [ ]:
# 利用可能な最新情報を取得するためのウェブ検索を実行
search_results = tavily_client.search(
    query="今日NYCで何が起こった？", max_results=5
)

この検索呼び出しは5つの結果を返します。各結果には、ウェブページのタイトル、URL、RAG目的のコンテンツスニペット、およびクエリとの一致度を示すセマンティックスコアが含まれます。

In [ ]:
# 結果を表示
for result in search_results["results"]:
    print(result["title"])
    print(result["url"])
    print(result["content"])
    print(result["score"])
    print("\n")

Tavilyの動作を見るために、さまざまなAPIパラメータ設定を試してみましょう。幅広いトピックから特定の質問まで、すべてを試してみてください！結果の数、時間範囲、ドメインフィルターなどのパラメータを調整して、検索をカスタマイズできます。詳細については、検索[APIリファレンス](https://docs.tavily.com/documentation/api-reference/endpoint/search?utm_source=github&utm_medium=referral&utm_campaign=nir_diamant)と[ベストプラクティスガイド](https://docs.tavily.com/documentation/best-practices/best-practices-search?utm_source=github&utm_medium=referral&utm_campaign=nir_diamant)を参照してください。時間範囲フィルター、ドメインフィルターを適用し、`news`検索トピックを使用してみましょう。

In [ ]:
# 特定の時間範囲、ドメインフィルター、トピックフィルターを使用してウェブ検索を実行
search_results = tavily_client.search(
    query="Anthropicのモデルリリース？",
    max_results=5,
    time_range="month",
    include_domains=["techcrunch.com"],
    topic="news",
)

すべての結果が`techcrunch.com`からのもので、過去1か月に限定されていることに注目してください。`news`トピックを設定することで、検索は信頼できるサードパーティのニュースソースに焦点を当てています。

In [ ]:
# 結果を表示
for result in search_results["results"]:
    print(result["title"])
    print(result["url"])
    print(result["content"])
    print("\n")

## 抽出 📄 

次に、Tavilyの抽出エンドポイントを使用して、前の検索結果のURLを使用して各ページの完全なコンテンツ（つまり、`raw_content`）を取得します。検索からの短いコンテンツスニペットだけを使用するのではなく、これにより各ページの全文にアクセスできます。効率のために、抽出エンドポイントは1回の呼び出しで最大20個のURLを一度に処理できます。詳細については、抽出[APIリファレンス](https://docs.tavily.com/documentation/api-reference/endpoint/extract?utm_source=github&utm_medium=referral&utm_campaign=nir_diamant)と[ベストプラクティスガイド](https://docs.tavily.com/documentation/best-practices/best-practices-extract?utm_source=github&utm_medium=referral&utm_campaign=nir_diamant)を参照してください。

In [ ]:
# 検索結果のURLからフルページコンテンツを抽出
extract_results = tavily_client.extract(
    urls=[result["url"] for result in search_results["results"]],
    # extract_depth="advanced", #動的コンテンツ、埋め込みメディア、または構造化データを含む複雑なウェブページに対して高度な抽出深度を使用するにはコメントを解除
)

以前に表示された短いコンテンツスニペットよりもはるかに詳細で完全な情報を提供する生のコンテンツを見てみましょう。生のコンテンツをLLMへの入力として使用する場合は、モデルのコンテキストウィンドウの制限を考慮することを忘れないでください。

In [ ]:
# 結果を表示
for result in extract_results["results"]:
    print(result["url"])
    print(result["raw_content"])
    print("\n")

生のページコンテンツを返すために抽出エンドポイントを使用する代わりに、`include_raw_content=True`パラメータを使用して検索エンドポイントで検索と抽出エンドポイントを1回のAPI呼び出しに組み合わせることができます。

In [ ]:
# ライブコンテンツ抽出を伴うウェブ検索を実行
search_results = tavily_client.search(
    query="Anthropicのモデルリリース？",
    max_results=1,
    include_raw_content=True,
)

各検索結果には、ウェブページのタイトル、URL、セマンティックスコア、コンテンツスニペット、および完全な生のコンテンツが含まれるようになりました。Tavilyの柔軟でモジュラーなAPIは、モデルサイズに関係なく、幅広いエージェントシステムの構築をサポートします。

In [ ]:
# 結果を表示
for result in search_results["results"]:
    print(result["url"])
    print(result["content"])
    print(result["score"])
    print(result["raw_content"])
    print("\n")

## クロール 🕸️ 

次に、Tavilyを使用してウェブページをクロールし、すべてのリンクを抽出しましょう。ウェブクロールは、ハイパーリンクをたどってウェブサイトを自動的にナビゲートし、多数のウェブページとURLを発見するプロセスです（Wikipediaのうさぎ穴に落ちるようなもの🐇—ページからページへクリックし、相互接続されたトピックに深く潜っていく）。自律的なウェブエージェントにとって、この機能は検索では取得が困難な深いウェブデータにアクセスするために不可欠です。詳細については、クロール[APIリファレンス](https://docs.tavily.com/documentation/api-reference/endpoint/crawl?utm_source=github&utm_medium=referral&utm_campaign=nir_diamant)と[ベストプラクティスガイド](https://docs.tavily.com/documentation/best-practices/best-practices-crawl?utm_source=github&utm_medium=referral&utm_campaign=nir_diamant)を参照してください。

Tavilyウェブサイトをクロールしてすべてのネストされたページを収集することから始めましょう。

In [ ]:
# Tavilyウェブサイトをクロール
crawl_results = tavily_client.crawl(url="tavily.com")

すべてのネストされたURLが表示されます。

In [ ]:
# 結果を表示
for result in crawl_results["results"]:
    print(result["url"])

クロールエンドポイントは各URLの生のページコンテンツも返します。

In [ ]:
# 結果を表示
for result in crawl_results["results"]:
    print(result["url"])
    print(result["raw_content"])
    print("\n")

リンクだけに興味がある場合（ページ全体のコンテンツなし）は、Mapエンドポイントを使用します。サイトからすべてのリンクを取得するための、より高速でコスト効率の良い方法です。

In [ ]:
# Tavilyウェブサイトをマップ
map_results = tavily_client.map(url="tavily.com")

結果を見てみましょう。この場合はリンクのみが含まれています。

In [ ]:
# 結果を表示
map_results

クロール/マップエンドポイントの`instructions`パラメータは、自然言語の指示を使用してウェブクロールをガイドできる強力な機能です。

In [ ]:
# 自然言語の指示でTavilyウェブサイトをマップ
guided_map_results = tavily_client.map(
    url="tavily.com", instructions="開発者ドキュメントのみを見つける"
)

これで、結果にはTavilyウェブページの開発者ドキュメントのみが含まれます。

In [ ]:
guided_map_results

さまざまなURLで実験して、Tavilyがさまざまなウェブサイトの構造をどのようにマップするかを確認してください。これをあなたのエージェントシステムにどのように統合しますか...🤔？

## まとめと次のステップ
 
このチュートリアルでは、以下を学びました：
- Tavily APIを使用したリアルタイムウェブ検索の実行方法
- ウェブページからのコンテンツ抽出方法
- ウェブサイトのクロールとマップによるリンクと情報の収集方法
- ターゲットデータ抽出のための自然言語指示によるクロールのガイド方法
 
これらの基礎的なスキルにより、エージェントは最新のウェブ情報にアクセスして活用できるようになり、より強力でコンテキストを理解できるものになります。[プレイグラウンド](https://app.tavily.com/playground?utm_source=github&utm_medium=referral&utm_campaign=nir_diamant)でTavily APIを自由に実験し、ユースケースに最適化するために[ベストプラクティスガイド](https://docs.tavily.com/documentation/best-practices/best-practices-search?utm_source=github&utm_medium=referral&utm_campaign=nir_diamant)を読んでください。
 
**次のステップに進む準備はできましたか？**  
**チュートリアル#2：自律的に検索、抽出、クロールできるウェブエージェントの構築**では、これらの機能を組み合わせて完全に自律的なウェブエージェントを構築します。このエージェントは推論し、いつ検索、クロール、または抽出するかを決定し、ウェブデータをワークフローに統合できます—すべてTavilyによって駆動されます。
 
[👉 **チュートリアル#2に進む！**](./web-agent-tutorial.ipynb)